In [ ]:
import numpy as np
import keras
import os
import pandas as pd
from keras import callbacks
import cv2
#from scipy import ndimage as ndi
import PIL
from PIL import Image
from sklearn.utils import shuffle
from os import listdir
from keras.models import Sequential,load_model
#from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.optimizers import Adam
#from keras.preprocessing import image
from sklearn.metrics import classification_report
#from keras import regularizers
#from skimage.color import rgb2gray
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
directory_root = "../input/plantdisease/plantvillage/PlantVillage/"
print(os.listdir("../input/plantdisease/plantvillage/PlantVillage/"))
print(len(listdir(directory_root)))

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print(f"Processing {plant_folder} ...")
      

        for image in plant_disease_folder_list:
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(image_directory)
                label_list.append(plant_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

In [ ]:
img_data = pd.DataFrame({'image_path':image_list,'label':label_list})
print(img_data.head())
print(len(img_data))

In [ ]:

img_data["labels_integer"] = None

index_labels_integer = img_data.columns.get_loc("labels_integer")

index_species = img_data.columns.get_loc("label")

k = 0 
for i in range(len(img_data)):
    if i == 0:
        img_data.iloc[i, index_labels_integer] = k 
    if i > 0:
        if img_data.iloc[i-1, index_species] == img_data.iloc[i, index_species]:
            img_data.iloc[i, index_labels_integer] = k
        else:
            k += 1
            img_data.iloc[i, index_labels_integer] = k
#img_data[img_data["label"].unique()]
#print(np.unique(img_data['labels_integer']))

In [ ]:
img_data.drop_duplicates(subset = ["labels_integer"])

In [ ]:
img_data= shuffle(img_data)
list_vectors = []

for image_path in img_data.image_path:
   
    img = Image.open(image_path)
    size = (64, 64)
    img = img.resize(size, PIL.Image.ANTIALIAS)  
    img_array = np.array(img)
    list_vectors.append(img_array)
    
print(len(list_vectors))

In [ ]:
X = np.stack((list_vectors))
Y =  img_data['labels_integer']
print(X.shape)

In [ ]:
X = X/255
Y_one_hot = keras.utils.to_categorical(Y, num_classes=15)
print(Y_one_hot[0:15])
print(Y.shape, Y_one_hot.shape)

In [ ]:
np.savez("x_images_arrayscnn", X)
np.savez("y_numeric_labelscnn", Y_one_hot)

In [ ]:
x_npz = np.load("x_images_arrayscnn.npz")
X = x_npz['arr_0']

y_npz = np.load("y_numeric_labelscnn.npz")
Y_one_hot = y_npz['arr_0']

print(X.shape)

In [ ]:
split_train = 0.8 
split_val = 0.9
index_train = int(split_train*len(X))
index_val = int(split_val*len(X))

X_train = X[:index_train]
X_val = X[index_train:index_val]
X_test = X[index_val:]

Y_train = Y_one_hot[:index_train]
Y_val = Y_one_hot[index_train:index_val]
Y_test = Y_one_hot[index_val:]


print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]) 
num_classes = 15

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer=keras.optimizers.Adam(lr=0.0003, decay=1e-8),loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
 best_model_file = "disease_detection.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model...')
results = model.fit(X_train, Y_train, epochs=50, batch_size=64, validation_data=(X_val, Y_val), callbacks=[best_model])
print('Traing finished.')


In [ ]:
scores=model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
Y_test = np.argmax(Y_test,axis=1)
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

In [ ]:
acc = results.history['accuracy']
val_acc = results.history['val_accuracy']
loss = results.history['loss']
val_loss = results.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()